In [1]:
import sys
sys.path.append('/home/ngoel/programs/vampyr/build/lib/python3.13/site-packages')
import numpy as np
from vampyr import vampyr3d as vp

In [2]:
class Molecule:
    def __init__(self, atoms = [], path = "./initial_guess/"):
        self.atoms = atoms
        self.path = path
    def getAtoms(self):
        return self.atoms
    def getPath(self):
        return self.path

class Atom:
    def __init__(self, coords = (0.0, 0.0, 0.0), charge = 1):
        self.coords = coords
        self.charge = charge
    def getCoords(self):
        return self.coords
    def getCharge(self):
        return self.charge

In [ ]:
class HelmholtzOperator:
    def __init__(self, mra, alpha, precision):
        self.mra = mra
        self.alpha = alpha
        self.precision = precision
        self.operators = []

        mu = [np.sqrt(-2.0 * l) if l < 0 else 1.0 for l in self.alpha]
        for m in mu:
            self.operators.append(vp.HelmholtzOperator(mra = self.mra, exp = m, prec = self.precision))

    def __call__(self, Psi):
        return np.array([self.operators[i](Psi[i]) for i in range(len(Psi))])

class CoulombOperator:
    def __init__(self, mra, Psi, precision):
        self.mra = mra
        self.Psi = Psi
        self.precision = precision

        poisson = vp.PoissonOperator(mra, precision)
        rho = self.Psi[0]**2
        for i in range(1, len(self.Psi)):
            rho += self.Psi[i]**2
        self.potential = 4.0 * np.pi * poisson(rho).crop(self.precision / 10)
        # self.potential = 4.0 * np.pi * poisson(rho)

    def __call__(self, Phi):
        return np.array([(self.potential * phi).crop(self.precision / 10) for phi in Phi])
        # return np.array([(self.potential * phi) for phi in Phi])

class ExchangeOperator:
    def __init__(self, mra, Psi, precision):
        self.mra = mra
        self.Psi = Psi
        self.precision = precision
        self.poisson = vp.PoissonOperator(mra, precision)

    def __call__(self, Phi):
        out = []
        for phi in Phi:
            tmp = (self.Psi[0] * self.poisson(phi * self.Psi[0])).crop(self.precision / 10)
            # tmp = (self.Psi[0] * self.poisson(phi * self.Psi[0]))
            for i in range (1, len(self.Psi)):
                tmp += (self.Psi[i] * self.poisson(phi * self.Psi[i])).crop(self.precision / 10)
                # tmp += (self.Psi[i] * self.poisson(phi * self.Psi[i]))
            out.append(tmp)
        return 4.0 * np.pi * np.array([phi.crop(self.precision / 10) for phi in out])
        # return 4.0 * np.pi * np.array([phi for phi in out])

class KineticOperator:
    def __init__(self, mra, precision):
        self.mra = mra
        self.precision = precision
        self.derivative = vp.ABGVDerivative(mra, 0.5, 0.5)

    def laplacian(self, phi):
        return self.derivative(self.derivative(phi, 0), 0) + self.derivative(self.derivative(phi, 1), 1) + self.derivative(self.derivative(phi, 2), 2)

    def __call__(self, Phi):
        return np.array([(-0.5 * self.laplacian(phi)).crop(self.precision / 10) for phi in Phi])
        # return np.array([(-0.5 * self.laplacian(phi)) for phi in Phi])

class NuclearOperator:
    def __init__(self, molecule, mra, precision):
        self.molecule = molecule
        self.mra = mra
        self.precision = precision

        P = vp.ScalingProjector(mra, precision)
        nucFunc = NucFunc(molecule)
        self.potential = P(nucFunc)

    def __call__(self, Phi):
        return np.array([(self.potential * phi).crop(self.precision / 10) for phi in Phi])
        # return np.array([(self.potential * phi) for phi in Phi])

class NucFunc:
    def __init__(self, molecule):
        self.molecule = molecule
        self.coords = np.array([atom.getCoords() for atom in self.molecule.getAtoms()])
        self.charges = np.array([atom.getCharge() for atom in self.molecule.getAtoms()])
        self.nAtoms = len(self.molecule.getAtoms())
    
    def __call__(self, r):
        r = np.asarray(r, dtype=float)
        out = -sum(self.charges / np.linalg.norm(self.coords - r, axis=1))
        return float(out)

In [4]:
# SCF Loop
def runSCF(mra, molecule, Phi, F, precision, threshold, maxIter = 100):
    energies = []
    updates = []

    V_nuc = NuclearOperator(molecule, mra, precision)
    J = CoulombOperator(mra, Phi, precision)
    K = ExchangeOperator(mra, Phi, precision)
    V = V_nuc(Phi) + 2 * J(Phi) - K(Phi)
    Kin = KineticOperator(mra, precision)
    F = calc_overlap(Phi, V + Kin(Phi), diag = False)

    iteration = 0
    while True:
        Lambda = np.diag(F)
        G = HelmholtzOperator(mra, Lambda, precision)
        Phi_new = -2 * G(V + (np.diag(Lambda) - F) @ Phi)
        dPhi = Phi_new - Phi
        update = np.array([phi.norm() for phi in dPhi])
        updates.append(update)
        Phi = loewdinOrthogonalization(Phi_new)

        # F, V, J_n, K_n = calculateFock(mra, Phi, V_nuc, precision)
        J = CoulombOperator(mra, Phi, precision)
        K = ExchangeOperator(mra, Phi, precision)
        V_n = V_nuc(Phi)
        J_n = J(Phi)
        K_n = K(Phi)
        V = V_n + 2 * J_n - K_n
        Kin = KineticOperator(mra, precision)
        F = calc_overlap(Phi, V + Kin(Phi), diag = False)

        energy = calculateEnergy(Phi, F, V, J_n, K_n)
        energies.append(energy)
        print(iteration, " |  E_tot:", energy["$E_{tot}$"], " |  E_HF:", energy["$E_{tot}$"] + calculateNuclearEnergy(molecule), " |  dPhi:", max(update))
        iteration += 1

        if max(update) < threshold or iteration > maxIter:
            break
    return Phi, energies, np.array(updates)

def initialGuess(mra, molecule, prec):
    P = vp.ScalingProjector(mra, prec)

    Phi = []
    atoms = molecule.getAtoms()
    atoms.sort(key=lambda x: x.getCharge())
    nOrbs_total = int((np.array([atom.getCharge() for atom in atoms]).sum()) / 2.0)
    nOrbs_current = 0
    for a in range(len(atoms)):
        nOrbs = int(max(atoms[a].getCharge() / 2, 1))
        if a == len(atoms) - 1:
            nOrbs = int(nOrbs_total - nOrbs_current)
        nOrbs_current += nOrbs
        for i in range (1, nOrbs + 1):
            R = atoms[a].getCoords()
            def f(r):
                return np.exp(-np.linalg.norm(np.array(r) - np.array(R)) / i)
            phi = P(f)
            phi.normalize()
            Phi.append(phi)
    Phi = np.array(Phi)
    return loewdinOrthogonalization(Phi)

def loewdinOrthogonalization(Phi):
    S = calc_overlap(Phi, Phi, diag = True)
    sigma, U = np.linalg.eigh(S)
    Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()
    Phi = Sm5 @ Phi
    for phi in Phi:
        phi.normalize()
    return Phi

def calc_overlap(Psi, Phi, diag = False):
    S = np.zeros((len(Psi), len(Phi)))
    for i in range(len(Psi)):
        if diag:
            for j in range(i, len(Phi)):
                S[i, j] = vp.dot(Psi[i], Phi[j])
        else:
            for j in range(len(Phi)):
                S[i, j] = vp.dot(Psi[i], Phi[j])
    if diag:
        S += S.transpose()
        S[np.diag_indices_from(S)] /= 2.0
    return S

def calculateEnergy(Phi, F, V, J, K):
    V_mat = calc_overlap(Phi, V, diag=True)
    J_mat = calc_overlap(Phi, J, diag=True)
    K_mat = calc_overlap(Phi, K, diag=True)

    e = 2.0 * F.trace()
    E_en = 2.0 * V_mat.trace()
    E_el = 2.0 * J_mat.trace() - K_mat.trace()
    E_tot = e - E_el
    E_kin = E_tot - E_en - E_el

    return {
        "$E_{orb}$": e,
        "$E_{en}$": E_en,
        "$E_{el}$": E_el,
        "$E_{kin}$": E_kin,
        "$E_{tot}$": E_tot
    }

def calculateNuclearEnergy(molecule):
    atoms = molecule.getAtoms()
    energy = 0.0
    for i in range(len(atoms)):
        R = atoms[i].getCoords()
        Z = atoms[i].getCharge()
        for j in range(i + 1, len(atoms)):
            Rj = atoms[j].getCoords()
            Zj = atoms[j].getCharge()
            energy += Z * Zj / np.linalg.norm(np.array(R) - np.array(Rj))
    return energy


### KAIN

In [5]:
def setupLinearSystem(phi_history, f_history):
    lenHistory = len(phi_history) - 1
    A = np.zeros((lenHistory, lenHistory))
    b = np.zeros((lenHistory))
    for i in range(lenHistory):
        dPhi = phi_history[i] - phi_history[-1]
        # b_i = < phi_i - phi_n | f_n >
        b[i] = vp.dot(dPhi, f_history[-1])
        for j in range(lenHistory):
            # A_ij = < phi_i - phi_n | f_j - f_n >
            A[i, j] -= vp.dot(dPhi, f_history[j] - f_history[-1])
    # solve Ax = b
    return np.linalg.solve(A, b)

def expandSolution(mra, Phi, f_history, F, V, V_nuc, precision, kain):
    # Phi_n: current orbitals
    Phi_n = [phi[-1] for phi in Phi]
    # obatain new orbitals from current guess
    Lambda = np.diag(F)
    G = HelmholtzOperator(mra, Lambda, precision)
    Phi_new = -2 * G(V + (np.diag(Lambda) - F) @ Phi_n)
    # orthogonalize new orbitals and obtain new orbital update
    Phi_new = loewdinOrthogonalization(Phi_new)
    dPhi = Phi_new - Phi_n
    
    update = []
    if kain:
        for i in range(len(Phi)):
            # if kain is used, add new orbitals and updates to history
            Phi[i].append(Phi_new[i])
            f_history[i].append(dPhi[i])
            print("nHistory before lin System:", len(Phi[i]))
            # solve Ax = b
            x = setupLinearSystem(Phi[i], f_history[i])
            print(x)
            # calculate 
            delta = dPhi[i]
            tmp = Phi[i][-1] + f_history[i][-1] # for efficiency
            for j in range(len(x)):
                linSys = Phi[i][j] + f_history[i][j] - tmp
                delta += x[j] * linSys
            # Phi_new[i] = Phi[i][-1] + delta
            Phi_new[i] += delta
            update.append(delta.norm())
        
        Phi_new = loewdinOrthogonalization(Phi_new) # not sure if needed again
    for i in range(len(Phi_new)):
        Phi_new[i].normalize()
        dPhi[i] = Phi_new[i] - Phi_n[i]
        if kain:
            del Phi[i][-1]
            del f_history[i][-1]
        else:
            update.append(dPhi[i].norm())
        Phi[i].append(Phi_new[i])
        f_history[i].append(dPhi[i])

    Phi_np1 = [x[-1] for x in Phi]
    F, V, J_n, K_n = calculateFock(mra, Phi_np1, V_nuc, precision)

    return Phi, f_history, F, V, J_n, K_n, update

def calculateFock(mra, Phi_n, V_nuc, precision):
    J = CoulombOperator(mra, Phi_n, precision)
    K = ExchangeOperator(mra, Phi_n, precision)
    V_n = V_nuc(Phi_n)
    J_n = J(Phi_n)
    K_n = K(Phi_n)
    V = V_n + 2 * J_n - K_n
    Kin = KineticOperator(mra, precision)
    F = calc_overlap(Phi_n, V + Kin(Phi_n), diag = False)
    print("F: ", F)
    return F, V, J_n, K_n

def runKAINSCF(mra, molecule, Phi, f_history, F, precision, threshold, maxIter = 100, kain_start = 3, kain_history = 5):
    energies = []
    updates = []

    Phi_n = [phi[-1] for phi in Phi]
    V_nuc = NuclearOperator(molecule, mra, precision)
    F, V, J_n, K_n = calculateFock(mra, Phi_n, V_nuc, precision)
    
    iteration = 0
    while True:
        kain = iteration >= kain_start
        Phi, f_history, F, V, J_n, K_n, update = expandSolution(mra, Phi, f_history, F, V, V_nuc, precision, kain)

        Phi_np1 = [x[-1] for x in Phi]
        energy = calculateEnergy(Phi_np1, F, V, J_n, K_n)
        energies.append(energy)
        print(iteration, " |  E_tot:", energy["$E_{tot}$"], " |  E_HF:", energy["$E_{tot}$"] + calculateNuclearEnergy(molecule), " |  dPhi:", max(update))

        if not kain:
            if len(Phi[0]) > 1:
                for i in range(len(Phi)):
                    del Phi[i][0]
                    del f_history[i][0]
        else:
            if len(Phi[0]) > kain_history:
                for i in range(len(Phi)):
                    del Phi[i][0]
                    del f_history[i][0]


        iteration += 1
        print("nHistory:", len(Phi[0]))

        if max(update) < threshold or iteration > maxIter:
            break
    return Phi, energies, np.array(updates)

def initialGuessKAIN(mra, molecule, precision):
    P = vp.ScalingProjector(mra, precision)

    Phi = []
    atoms = molecule.getAtoms()
    atoms.sort(key=lambda x: x.getCharge())
    nOrbs_total = int((np.array([atom.getCharge() for atom in atoms]).sum()) / 2.0)
    nOrbs_current = 0
    for a in range(len(atoms)):
        nOrbs = int(max(atoms[a].getCharge() / 2, 1))
        if a == len(atoms) - 1:
            nOrbs = int(nOrbs_total - nOrbs_current)
        nOrbs_current += nOrbs
        for i in range (1, nOrbs + 1):
            R = atoms[a].getCoords()
            def f(r):
                return np.exp(-np.linalg.norm(np.array(r) - np.array(R)) / i)
            phi = P(f)
            phi.normalize()
            Phi.append(phi)
    Phi = np.array(Phi)
    Phi = loewdinOrthogonalization(Phi)

    V_nuc = NuclearOperator(molecule, mra, precision)
    F, V, J_n, K_n = calculateFock(mra, Phi, V_nuc, precision)
    Lambda = np.diag(F)
    G = HelmholtzOperator(mra, Lambda, precision)
    Phi_new = -2 * G(V + (np.diag(Lambda) - F) @ Phi)
    dPhi = Phi_new - Phi
    Phi_new = loewdinOrthogonalization(Phi_new)
    f_history = [[phi] for phi in Phi]
    Phi = [[phi] for phi in Phi]  # Each orbital history is a list
    for i in range(len(Phi_new)):
        f_history[i].append(dPhi[i])
        Phi_new[i].normalize()
        Phi[i].append(Phi_new[i])
        del f_history[i][0]
        del Phi[i][0]
    
    # F, V, J_n, K_n = calculateFock(mra, Phi_new, V_nuc, precision)
    # print("F: ", F)
    # energy = calculateEnergy(Phi_new, F, V, J_n, K_n)
    # print("-1  |  E_tot:", energy["$E_{tot}$"], " |  E_HF:", energy["$E_{tot}$"] + calculateNuclearEnergy(molecule))

    return Phi, f_history, F

### KAIN MRCHEM

In [ ]:
def setupLinearSystem(phi_history, f_history): # checked, exactly the same as Quentin's
    # lenHistory only includes previous steps, not current
    lenHistory = len(phi_history) - 1
    A = np.zeros((lenHistory, lenHistory))
    b = np.zeros((lenHistory))
    for i in range(lenHistory):
        dPhi = phi_history[i] - phi_history[-1]
        # b_i = < phi_i - phi_n | f_n >
        b[i] = vp.dot(dPhi, f_history[-1])
        for j in range(lenHistory):
            # A_ij = < phi_i - phi_n | f_j - f_n >
            A[i, j] -= vp.dot(dPhi, f_history[j] - f_history[-1])
    # solve Ax = b
    return np.linalg.solve(A, b)

def expandSolution(mra, Phi_np1, Phi, f_history, F, V, V_nuc, precision, iteration, kain_start, kain_history):
    # Phi_n: current orbitals
    # obatain new orbitals from current guess
    Lambda = np.diag(F)
    G = HelmholtzOperator(mra, Lambda, precision)
    Phi_np1 = -2 * G(V + (np.diag(Lambda) - F) @ Phi_np1)
    # orthogonalize new orbitals and obtain new orbital update
    Phi_np1 = loewdinOrthogonalization(Phi_np1) # maybe not?
    print("new Phi obtained")
    
    update = []
    for i in range(len(Phi_np1)):
        # add current orbital and new orbital update to history
        f_history[i].append(Phi_np1[i] - Phi[i][-1])
        # solve Ax = b
        x = setupLinearSystem(Phi[i], f_history[i])
        # calculate correction to orbital update
        delta = f_history[i][-1]
        # dPhi[i] = sum_j (Phi[i][j] + f_history[i][j] - Phi[i][-1] - f_history[i][-1]) * x[j]
        tmp = Phi[i][-1] + f_history[i][-1] # for efficiency
        for j in range(len(x)):
            linSys = Phi[i][j] + f_history[i][j] - tmp
            delta += x[j] * linSys
        # calculate convergence measure
        update.append(delta.norm())
        # update current orbitals with new correction
        Phi_np1[i] = Phi[i][-1] + delta
        Phi_np1[i].normalize()
    print("corrections applied")

    Phi_np1 = loewdinOrthogonalization(Phi_np1)
    for i in range(len(Phi_np1)):
        Phi[i].append(Phi_np1[i])
        if iteration < kain_start or len(Phi[i]) > kain_history:
            del Phi[i][0]
            del f_history[i][0]

    #     # add new orbital to history
    #     Phi[i].append(Phi_np1[i])
    #     # make sure, history does not exceed max history or 1 if kain is not invoked (yet)
    #     if iteration < kain_start or len(Phi[i]) > kain_history:
    #         del Phi[i][0]
    #         del f_history[i][0]

    # Phi_n = loewdinOrthogonalization(Phi_np1)
    # calculate new Fock matrix
    print("calculating new Fock")
    F, V, J_n, K_n = calculateFock(mra, Phi_np1, V_nuc, precision)
    # calculate new energy
    energy = calculateEnergy(Phi_np1, F, V, J_n, K_n)

    return Phi_np1, Phi, f_history, F, V, update, energy

def calculateFock(mra, Phi_n, V_nuc, precision):
    J = CoulombOperator(mra, Phi_n, precision)
    K = ExchangeOperator(mra, Phi_n, precision)
    V_n = V_nuc(Phi_n)
    J_n = J(Phi_n)
    K_n = K(Phi_n)
    V = V_n + 2 * J_n - K_n
    # for v in V:
    #     v.crop(precision / 10)
    Kin = KineticOperator(mra, precision)
    F = calc_overlap(Phi_n, V + Kin(Phi_n), diag = False)
    return F, V, J_n, K_n

def runKAINSCF(mra, molecule, precision, threshold, maxIter = 100, kain_start = 0, kain_history = 5):
    # calculate first initial guess and create a first history
    Phi_n, Phi, f_history, V_nuc = initialGuessKAIN(mra, molecule, precision, kain_history)
    energies = []
    updates = []

    F, V, J_n, K_n = calculateFock(mra, Phi_n, V_nuc, precision)
    
    iteration = 0
    while True:
        print(f"=============Iteration: {iteration}")
        # Here, we calculate new orbitals, apply KAIN and calculate the new Fock matrix
        Phi_n, Phi, f_history, F, V, update, energy = expandSolution(mra, Phi_n, Phi, f_history, F, V, V_nuc, precision, iteration, kain_start, kain_history)
        updates.append(update)
        energies.append(energy)
        print(iteration, " |  E_tot:", energy["$E_{tot}$"], " |  E_HF:", energy["$E_{tot}$"] + calculateNuclearEnergy(molecule), " |  dPhi:", max(update))
        print("E_orb:", energy["$E_{orb}$"], " E_en:", energy["$E_{en}$"], " E_el:", energy["$E_{el}$"], " E_kin:", energy["$E_{kin}$"])
        iteration += 1

        if max(update) < threshold or iteration > maxIter:
            break
    return Phi, energies, np.array(updates)

def initialGuessKAIN(mra, molecule, precision, kain_history):
    init_g_dir = molecule.getPath()
    Phi = []
    atoms = molecule.getAtoms()
    nOrbs_total = int((np.array([atom.getCharge() for atom in atoms]).sum()) / 2.0)
    for i in range(nOrbs_total):
        phi = vp.FunctionTree(mra)
        phi.loadTree(f"{init_g_dir}phi_p_scf_idx_{i}_re")
        Phi.append(phi)
    Phi_array = np.array(Phi)
    Phi = [[phi] for phi in Phi]
    f_history = [[] for i in range(len(Phi_array))]
    print("Initial guess loaded from disk.")

    # make a first step to obtain a minimal history
    V_nuc = NuclearOperator(molecule, mra, precision)
    print("Calculating initial Fock...")
    F, V, J_n, K_n = calculateFock(mra, Phi_array, V_nuc, precision)
    print("Obtaing first update...")
    Lambda = np.diag(F)
    G = HelmholtzOperator(mra, Lambda, precision)
    Phi_np1 = -2 * G(V + (np.diag(Lambda) - F) @ Phi_array)
    dPhi = Phi_np1 - Phi_array
    print("First update obtained.")
    for i in range(len(Phi_np1)):
        Phi_np1[i].normalize()
    Phi_np1 = loewdinOrthogonalization(Phi_np1)
    for i in range(len(Phi_np1)):
        f_history[i].append(dPhi[i])
        Phi[i].append(Phi_np1[i])
        if kain_history == 0:
            del f_history[i][0]
            del Phi[i][0]
    
    return Phi_np1, Phi, f_history, V_nuc

### Running

In [ ]:
precision = 1e-4
k = 5
box = [-20, 20]
mra = vp.MultiResolutionAnalysis(box, k)

# Water 1
atom1 = Atom(coords=(3.03640321, -0.125273197, 0.165537034), charge=8)
atom2 = Atom(coords=(1.20775692, 0.126394035, 0.040864271), charge=1)
atom3 = Atom(coords=(3.7449532, 1.653846757, 0.068992382), charge=1)
# Water 2
atom4 = Atom(coords=(-2.236881593, 0.079767042, -0.135775327), charge=8)
atom5 = Atom(coords=(-2.80936204, -0.932460622, -1.444139354), charge=1)
atom6 = Atom(coords=(-2.942869717, -0.602274003, 1.504521009), charge=1)
molecule = Molecule(atoms=[atom1, atom2, atom3, atom4, atom5, atom6])
# he1 = Atom(coords=(0.0, 0.0, 0.0), charge=2)
# he2 = Atom(coords=(3.0, 0.0, 0.0), charge=2)
# molecule = Molecule(atoms=[he1, he2])
# be1 = Atom(coords=(0.0, 0.0, 0.0), charge=4)
# molecule = Molecule(atoms=[he1])
Phi = initialGuess(mra, molecule, precision)
print("Initial orbitals:", Phi)
F = -calc_overlap(Phi, Phi, diag=True)

print(F)

Phi, energies, updates = runSCF(mra, molecule, Phi, F, precision, threshold=1e-3, maxIter=200)

In [ ]:
atom1 = Atom(coords=(0.0, 0.0, 0.7236307864), charge=8)
atom2 = Atom(coords=(-1.4179501563, 0.0, -0.3618153933), charge=1)
atom3 = Atom(coords=(1.4179501563, 0.0, -0.3618153933), charge=1)
molecule = Molecule(atoms=[atom1, atom2, atom3])
print(calculateNuclearEnergy(molecule))

9.31262093870235


In [ ]:
precision = 1e-4
k = 5
box = [-20, 20]
mra = vp.MultiResolutionAnalysis(box, k)

# Water 1
atom1 = Atom(coords=(3.03640321, -0.125273197, 0.165537034), charge=8)
atom2 = Atom(coords=(1.20775692, 0.126394035, 0.040864271), charge=1)
atom3 = Atom(coords=(3.7449532, 1.653846757, 0.068992382), charge=1)
# Water 2
# atom4 = Atom(coords=(-2.236881593, 0.079767042, -0.135775327), charge=8)
# atom5 = Atom(coords=(-2.80936204, -0.932460622, -1.444139354), charge=1)
# atom6 = Atom(coords=(-2.942869717, -0.602274003, 1.504521009), charge=1)
molecule = Molecule(atoms=[atom1, atom2, atom3])
# he1 = Atom(coords=(0.0, 0.0, 0.0), charge=2)
# he2 = Atom(coords=(3.0, 0.0, 0.0), charge=2)
# molecule = Molecule(atoms=[he1, he2])
# be1 = Atom(coords=(0.0, 0.0, 0.0), charge=4)
# molecule = Molecule(atoms=[he1])
Phi = initialGuess(mra, molecule, precision)
print("Initial orbitals:", Phi)
F = -calc_overlap(Phi, Phi, diag=True)

print(F)

Phi, energies, updates = runSCF(mra, molecule, Phi, F, precision, threshold=1e-3, maxIter=10)
Phi, energies, updates = runKAINSCF(mra, molecule, [Phi], F, precision, threshold=1e-3, maxIter=10)

Initial orbitals: [<vampyr._vampyr.vampyr3d.FunctionTree object at 0x112e22270>
[[-1.00000000e+00 -3.41225795e-16  6.44265043e-16 -2.24066690e-15
   3.09015123e-15]
 [-3.41225795e-16 -1.00000000e+00  3.98604485e-15 -6.58785723e-15
   6.30863702e-15]
 [ 6.44265043e-16  3.98604485e-15 -1.00000000e+00 -3.06487569e-15
   6.99776062e-16]
 [-2.24066690e-15 -6.58785723e-15 -3.06487569e-15 -1.00000000e+00
  -4.82642853e-15]
 [ 3.09015123e-15  6.30863702e-15  6.99776062e-16 -4.82642853e-15
  -1.00000000e+00]]
0  |  E_tot: -35.284745019600635  |  E_HF: -26.450777547248826  |  dPhi: 1.8675114827481618
1  |  E_tot: -35.284745019600635  |  E_HF: -26.450777547248826  |  dPhi: 1.8022699419857755
2  |  E_tot: -35.284745019600635  |  E_HF: -26.450777547248826  |  dPhi: 1.8022699419857755
3  |  E_tot: -35.284745019600635  |  E_HF: -26.450777547248826  |  dPhi: 1.8022699419857755
4  |  E_tot: -35.284745019600635  |  E_HF: -26.450777547248826  |  dPhi: 1.8022699419857755
5  |  E_tot: -35.284745019600635  

### KAIN running

In [ ]:
# Set up variables for VAMPyR
precision = 1e-4
world = vp.BoundingBox(corner = [-1]*3, nboxes = [2]*3, scaling = [1.0]*3, scale = -4)
mra = vp.MultiResolutionAnalysis(order = 8, box = world)

# Water 1
atom1 = Atom(coords = (3.03640321, -0.125273197, 0.165537034), charge = 8)
atom2 = Atom(coords = (1.20775692, 0.126394035, 0.040864271), charge = 1)
atom3 = Atom(coords = (3.7449532, 1.653846757, 0.068992382), charge = 1)
molecule = Molecule(atoms = [atom1, atom2, atom3], path="./water_4/")
# Water 2
# atom4 = Atom(coords = (-2.236881593, 0.079767042, -0.135775327), charge = 8)
# atom5 = Atom(coords = (-2.80936204, -0.932460622, -1.444139354), charge = 1)
# atom6 = Atom(coords = (-2.942869717, -0.602274003, 1.504521009), charge = 1)
# molecule = Molecule(atoms = [atom1, atom2, atom3, atom4, atom5, atom6], path="./diwater_5/")
# he1 = Atom(coords = (0.0, 0.0, 1.5), charge = 2)
# he2 = Atom(coords = (0.0, 0.0, -1.5), charge = 2)
# molecule = Molecule(atoms = [he1, he2], path = "./he2_4/")

Phi, energies, updates = runKAINSCF(mra, molecule, precision, threshold = precision*10, maxIter = 100, kain_start = 0, kain_history = 5)
print(updates)

Initial guess loaded from disk.
Calculating initial Fock...
Obtaing first update...
First update obtained.
=============Iteration: 0
new Phi obtained
corrections applied
calculating new Fock
0  |  E_tot: -189.106700739399  |  E_HF: -152.12327578950237  |  dPhi: 0.040064986601170464
E_orb: -94.75337715633839  E_en: -245.96795012272827  E_el: 94.3533235830606  E_kin: -37.492074199731334
=============Iteration: 1
new Phi obtained
corrections applied
calculating new Fock
1  |  E_tot: -189.11163154237312  |  E_HF: -152.1282065924765  |  dPhi: 0.011945075662997236
E_orb: -94.73419079358165  E_en: -246.4568890109777  E_el: 94.37744074879149  E_kin: -37.0321832801869
=============Iteration: 2
new Phi obtained
corrections applied
calculating new Fock
2  |  E_tot: -189.11241836461656  |  E_HF: -152.12899341471996  |  dPhi: 0.00810288732875025
E_orb: -94.7999756203202  E_en: -246.64564060802473  E_el: 94.31244274429636  E_kin: -36.77922050088819
=============Iteration: 3
new Phi obtained
correcti

In [ ]:
precision = 1e-4
k = 5
box = [-20, 20]
mra = vp.MultiResolutionAnalysis(box, k)

# Water 1
atom1 = Atom(coords = (3.03640321, -0.125273197, 0.165537034), charge = 8)
atom2 = Atom(coords = (1.20775692, 0.126394035, 0.040864271), charge = 1)
atom3 = Atom(coords = (3.7449532, 1.653846757, 0.068992382), charge = 1)
molecule = Molecule(atoms = [atom1, atom2, atom3])
# Water 2
# atom4 = Atom(coords = (-2.236881593, 0.079767042, -0.135775327), charge = 8)
# atom5 = Atom(coords = (-2.80936204, -0.932460622, -1.444139354), charge = 1)
# atom6 = Atom(coords = (-2.942869717, -0.602274003, 1.504521009), charge = 1)
# molecule = Molecule(atoms = [atom1, atom2, atom3])
he1 = Atom(coords = (0.0, 0.0, 0.0), charge = 2)
he2 = Atom(coords = (3.0, 0.0, 0.0), charge = 2)
# molecule = Molecule(atoms = [he1, he2])
# be1 = Atom(coords = (0.0, 0.0, 0.0), charge = 4)
# molecule = Molecule(atoms = [he1])
Phi = initialGuess(mra, molecule, precision)
F = -calc_overlap(Phi, Phi, diag = True)
print(F)

Phi, energies, updates = runSCF(mra, molecule, Phi, F, precision, threshold = precision*10, maxIter = 100)
print(updates)

[[-1.00000000e+00 -3.22118795e-16  6.51388657e-16 -2.20412616e-15
   3.08732470e-15]
 [-3.22118795e-16 -1.00000000e+00  4.02446309e-15 -6.64279661e-15
   6.27922250e-15]
 [ 6.51388657e-16  4.02446309e-15 -1.00000000e+00 -3.04357111e-15
   6.93401452e-16]
 [-2.20412616e-15 -6.64279661e-15 -3.04357111e-15 -1.00000000e+00
  -4.93444177e-15]
 [ 3.08732470e-15  6.27922250e-15  6.93401452e-16 -4.93444177e-15
  -1.00000000e+00]]
0  |  E_tot: -53.78188903356394  |  E_HF: -44.947921561212134  |  dPhi: 0.6106265163697274
1  |  E_tot: -70.30310991852039  |  E_HF: -61.46914244616858  |  dPhi: 0.4996674830331199
2  |  E_tot: -78.662917064513  |  E_HF: -69.82894959216118  |  dPhi: 0.4516610671920118
3  |  E_tot: -81.07139606301313  |  E_HF: -72.23742859066131  |  dPhi: 0.6859783983209574
4  |  E_tot: -82.56762991307392  |  E_HF: -73.7336624407221  |  dPhi: 0.7170851463972451
5  |  E_tot: -83.5930878866998  |  E_HF: -74.75912041434799  |  dPhi: 0.4085944276413843
6  |  E_tot: -83.93969078407954  |  E